In [18]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [20]:
def get_hashtag_network(df, hashtag: str):
    ''' returns a dataframe with the edges and timestamp from tweets feed (df)'''
    
    idx = (df.is_reply)  & (df.hashtags.swifter.apply(lambda x: hashtag in x))
    edges_df = df[idx][['user_id','reply_to_user_id', 'day']]
    
    edges_df = edges_df.dropna()
    edges_df = edges_df.drop_duplicates().reset_index(drop=True)
    edges_df = edges_df.rename(columns={'user_id': 'from', 'reply_to_user_id': 'to'})
    
    return edges_df


def get_strongly_gcc(G):
    """ get the giant strongly connected component of G""" 
    SGcc = max(nx.strongly_connected_components(G), key=len)
    SGcc = G.subgraph(SGcc)
    return SGcc

def get_weakly_gcc(G):
    """ get the giant weakly connected component of G"""  
    WGcc = max(nx.weakly_connected_components(G), key=len)
    WGcc = G.subgraph(WGcc)
    return WGcc
   

def plot_network(G, G_degree=None):
    ''' plot the graph with varying nodesize '''
    
    plt.figure(num=None, figsize=(15, 15), dpi=60, facecolor='w', edgecolor='k')
    pos = nx.spring_layout(G)
    
    if G_degree:
        node_size=[v * 10 for v in dict(G_degree).values()]
    else:
        node_size = 1
        
    nx.draw(G, pos, nodelist=dict(G_degree).keys(), node_size=node_size, width=0.5, alpha=0.5, edge_color='b')
    plt.axis('off')
    plt.show()

In [21]:
import re
text = "hello i am @monkey etc etc etc hi@outlook.com"
re.findall(r'\B@\w*[a-zA-Z]+\w*', text)


df = pd.read_csv('data/merged-and-pruned-dataset.csv')
df.head()
df.columns.tolist()

# countries_df = pd.read_csv(path/'Countries.csv', index_col=None, header=0)
# countries_df.head()

['status_id',
 'screen_name',
 'text',
 'source',
 'reply_to_status_id',
 'reply_to_screen_name',
 'is_retweet',
 'favourites_count',
 'retweet_count',
 'country_code',
 'followers_count',
 'friends_count',
 'verified',
 'lang']

In [22]:
# edges_df = get_hashtag_network(df, 'socialdistancing')


AttributeError: 'DataFrame' object has no attribute 'is_reply'